# Collection and resampling of data on cyptocurrencies

## Problem Statement
The cryptocurrency market has gained considerable popularity over the years, so more and more investors want to expand their portfolio by investing in these assets, however, data collection of these over time is scarce so it is very difficult for those who start to make an analysis based on this measure. What I will do in this project is to extract the trading data and transform it into OHLC (Open-High-Low-Close) format for a given frequency.

## 1. Obtaining data from Kraken (Exchanges)

In [36]:
import requests
import pandas as pd

In [37]:
# For this case, I will use the Ethereum (ETH) currency in U.S. dollars as a sample.

endpoint = 'https://api.kraken.com/0/public/Trades'
payLoad = {'pair': 'XETHZUSD'}
response = requests.get(url=endpoint, params=payLoad)
tradeData = response.json()
trades = tradeData['result']['XETHZUSD']

In [38]:
# Print the first 5 operations to check if everything is correct.

trades[:5]

[['1562.32000', '2.36796754', 1697037442.698231, 's', 'l', '', 45672353],
 ['1562.32000', '0.19094641', 1697037442.7509415, 's', 'l', '', 45672354],
 ['1562.01000', '0.17141963', 1697037442.7509916, 's', 'l', '', 45672355],
 ['1562.00000', '1.63764164', 1697037442.7510445, 's', 'l', '', 45672356],
 ['1562.33000', '0.17138447', 1697037442.8234644, 's', 'l', '', 45672357]]

In [39]:
tradesDF = pd.DataFrame.from_records(trades,
columns=['Price', 'Volume', 'Time', 'BuySell', 'MarketLimit', 'Misc', 'Others'])
tradesDF

,Price,Volume,Time,BuySell,MarketLimit,Misc,Others
0,1562.32000,2.36796754,1.697037e+09,s,l,,45672353
1,1562.32000,0.19094641,1.697037e+09,s,l,,45672354
2,1562.01000,0.17141963,1.697037e+09,s,l,,45672355
3,1562.00000,1.63764164,1.697037e+09,s,l,,45672356
4,1562.33000,0.17138447,1.697037e+09,s,l,,45672357
...,...,...,...,...,...,...,...
995,1553.76000,0.63408871,1.697047e+09,b,l,,45673348
996,1554.20000,0.01000000,1.697047e+09,b,l,,45673349
997,1554.19000,3.50000000,1.697047e+09,s,m,,45673350
998,1554.19000,2.37500000,1.697047e+09,s,m,,45673351


In [40]:
# Transforming the time column (from seconds to date & time)

tradesDF['Time'] = pd.to_datetime(tradesDF['Time'], unit='s')
tradesDF.set_index('Time', inplace=True)

tradesDF

,Price,Volume,BuySell,MarketLimit,Misc,Others
Time,,,,,,
2023-10-11 15:17:22.698231040,1562.32000,2.36796754,s,l,,45672353
2023-10-11 15:17:22.750941440,1562.32000,0.19094641,s,l,,45672354
2023-10-11 15:17:22.750991616,1562.01000,0.17141963,s,l,,45672355
2023-10-11 15:17:22.751044608,1562.00000,1.63764164,s,l,,45672356
2023-10-11 15:17:22.823464448,1562.33000,0.17138447,s,l,,45672357
...,...,...,...,...,...,...
2023-10-11 17:56:20.937641216,1553.76000,0.63408871,b,l,,45673348
2023-10-11 17:56:31.762122496,1554.20000,0.01000000,b,l,,45673349
2023-10-11 17:56:43.125687552,1554.19000,3.50000000,s,m,,45673350


In [41]:
# Obtaining more operations

tradeData["result"]["last"]

'1697047003125804130'

In [43]:
'1697041673354932896'
pd.to_datetime(int(tradeData["result"]["last"]),unit='ns')

Timestamp('2023-10-11 17:56:43.125804130')

In [48]:
import requests
import pandas as pd
import datetime
from datetime import timezone
import time 

def getKrakenTradeData(pair, startDate, endDate):
    endpoint = 'https://api.kraken.com/0/public/Trades'
    
    startTime = int(datetime.datetime.strptime(startDate, '%Y-%m-%d').
replace(tzinfo=timezone.utc).timestamp()*1000000000)
    endTime = int(datetime.datetime.strptime(endDate, '%Y-%m-%d').
replace(tzinfo=timezone.utc).timestamp()*1000000000)
    
    timeLoaded = startTime
    
    result = pd.DataFrame()
    
    while timeLoaded < endTime: 
        print(pd.to_datetime(timeLoaded, unit='ns').strftime('%Y-%m-%d %H:%M:%S'))
        payLoad = {'pair': pair,
                   'since': timeLoaded}
        
        response = requests.get(url=endpoint, params=payLoad)
        data = response.json()['result']
        tradesRaw = data[pair]
        timeLoaded = int(data['last'])
        
        tradeData = pd.DataFrame.from_records(tradesRaw, 
                            columns=['Price', 'Volume', 'Time', 'BuySell', 'MarketLimit', 'Misc', 'Others'])
        tradeData['Time'] = pd.to_datetime(tradeData['Time'], unit='s')
        
        result = result.append(tradeData)
        
        time.sleep(3)
        
        
    result.set_index('Time', inplace = True)
    result = result.loc[startDate:endDate+' 00:00:00']
        
    return result

In [49]:
trades

[['1562.32000', '2.36796754', 1697037442.698231, 's', 'l', '', 45672353],
 ['1562.32000', '0.19094641', 1697037442.7509415, 's', 'l', '', 45672354],
 ['1562.01000', '0.17141963', 1697037442.7509916, 's', 'l', '', 45672355],
 ['1562.00000', '1.63764164', 1697037442.7510445, 's', 'l', '', 45672356],
 ['1562.33000', '0.17138447', 1697037442.8234644, 's', 'l', '', 45672357],
 ['1562.33000', '0.17138447', 1697037442.874397, 's', 'l', '', 45672358],
 ['1562.00000', '0.36235836', 1697037442.87447, 's', 'l', '', 45672359],
 ['1561.58000', '0.01059833', 1697037443.0565, 's', 'l', '', 45672360],
 ['1561.57000', '0.23940167', 1697037443.0565581, 's', 'l', '', 45672361],
 ['1563.05000', '0.22014019', 1697037449.6975455, 'b', 'l', '', 45672362],
 ['1562.63000', '0.00165402', 1697037452.0835474, 'b', 'l', '', 45672363],
 ['1561.56000', '0.02259954', 1697037459.325272, 's', 'm', '', 45672364],
 ['1561.56000', '0.15423460', 1697037459.3253546, 's', 'm', '', 45672365],
 ['1561.56000', '7.30667649', 169

## 2. Resampling

In [57]:
# Sampling data for one hour

tradesDF.resample('1H')['Price'].agg(['first'])

,first
Time,
2023-10-11 15:00:00,1562.32000
2023-10-11 16:00:00,1559.26000
2023-10-11 17:00:00,1553.26000


In [54]:
# Transforming to OHLC

ohlc = tradesDF.resample('1H', label="left")['Price'].agg(['first', 'max', 'min', 'last']).rename(columns={"first": "Open", "max": "High", "min": "Low", "last": "Close"})

ohlc

,Open,High,Low,Close
Time,,,,
2023-10-11 15:00:00,1562.32000,1564.65000,1555.18000,1559.71000
2023-10-11 16:00:00,1559.26000,1560.62000,1551.17000,1553.56000
2023-10-11 17:00:00,1553.26000,1558.98000,1549.38000,1554.19000


In this project it was possible to extract the data, transform it and convert it to the desired format in a few lines of code, which opens a lot of doors when doing financial analysis and creating investment strategies.